### pytorch model 2
깊은 신경망을 이용한 classification 문제 해결
- 첫번째 레이어
    - 합성곱(in_channel=1, out_channel=32, kernel_size=3, strice=1, padding=1) + 활성화 함수(ReLU)
    - MaxPooling(kernel_size=2, stride=2)
- 두번째 레이어
    - 합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
    - MaxPooling(kernel_size=2, stride=2)
- 세번째 레이어
    - 합성곱(in_channel=64, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
    - MaxPooling(kernel_size=2, stride=2, padding=1)
- 네번째 레이어 : 전결합층(FC layer)  => 특성맵을 펼치는 역할
    - FC(Fully connected) layer + 활성화 함수 ReLU
- 다섯번째 레이어 : 전결합층(FC layer)
    - FC(Fully connected) layer + 활성화 함수 Softmax

(마지막 노드는 10개) -> 0~9까지의 정답을 가져야 하기 때문에 노드를 10개 설정.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# GPU 사용을 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [17]:
# 학습에 필요한 파라미터 설정
learning_rate = 0.001
training_epoch = 300
batch_size=100

In [18]:
# torchvision 데이터로드를 이용해 데이터셋을 정의
mnist_train=dsets.MNIST(root = 'MNIST_data/',
                       train = True,
                        transform=transforms.ToTensor(),
                       download=True)
mnist_test=dsets.MNIST(root = 'MNIST_data/',
                       train = False,
                        transform=transforms.ToTensor(),
                       download=True)

In [19]:
# 배치 크기를 지정하는 과정
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)
# 60000개의 데이터가 batch_size 100의 크기로 진행되기 때문에 total bach는 600이 됨

In [50]:
# 클래스를 이용하여 모델을 설계하기
class CNN(torch.nn.Module):
    def __init__(self):
        # 부모클래스 초기화(torch.nn.Module)
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        # 첫번째 레이어
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        # 두번째 레이어
        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        # 세번째 레이어
        self.layer3=torch.nn.Sequential(
            torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        
        self.layer4=torch.nn.Sequential(
            torch.nn.Conv2d(128,256,kernel_size=3,stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        
        self.layer5=torch.nn.Sequential(
            torch.nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        # 네번째 레이어
        self.fc1 = torch.nn.Linear(1*1*512, 1024, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer6=torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
        
        # 다섯번째 레이어
        self.fc2 = torch.nn.Linear(1024, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    # 순전파 전달을 위한 함수
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        out = self.layer5(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)
        out = self.layer6(out)
        out = self.fc2(out)
        return out

In [51]:
# CNN 모델을 정의
model = CNN().to(device)

In [52]:
# 비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [53]:
# 총배치의 수 확인하기
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
# 사용자가 지정한 배치의 수에 따라 총 배치의 수가 달라짐
# 배치 횟수 변경, 에폭 횟수 변경, 옵티마이저 변경, 손실함수 변경(비용함수) 등을 여러 방법으로 변경해 보며 
# 어떤 환경이 최적이었는지 판단하는 과정을 거쳐 사용에 있어서 최적의 모델을 만들어냄

총 배치의 수 : 600


In [54]:
# 학습 진행
for epoch in range(training_epoch):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += (cost/total_batch)
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch +1, avg_cost))

[Epoch:    1] cost = 0.224951357
[Epoch:    2] cost = 0.0541863367
[Epoch:    3] cost = 0.0384773165
[Epoch:    4] cost = 0.0299791507
[Epoch:    5] cost = 0.0274974313
[Epoch:    6] cost = 0.0232483707
[Epoch:    7] cost = 0.0194865949
[Epoch:    8] cost = 0.0180097334
[Epoch:    9] cost = 0.0143622551
[Epoch:   10] cost = 0.0137891825
[Epoch:   11] cost = 0.0133102797
[Epoch:   12] cost = 0.0109382877
[Epoch:   13] cost = 0.0126715666
[Epoch:   14] cost = 0.008771047
[Epoch:   15] cost = 0.0144241834
[Epoch:   16] cost = 0.00941325258
[Epoch:   17] cost = 0.0108594494
[Epoch:   18] cost = 0.00851859804
[Epoch:   19] cost = 0.0106856087
[Epoch:   20] cost = 0.00779218134
[Epoch:   21] cost = 0.00685833627
[Epoch:   22] cost = 0.00723296264
[Epoch:   23] cost = 0.0103283515
[Epoch:   24] cost = 0.00623814762
[Epoch:   25] cost = 0.00677241758
[Epoch:   26] cost = 0.00767571758
[Epoch:   27] cost = 0.00804779213
[Epoch:   28] cost = 0.00497926911
[Epoch:   29] cost = 0.00853557512
[Epoc

[Epoch:  237] cost = 0.00675693434
[Epoch:  238] cost = 0.00729490304
[Epoch:  239] cost = 0.00861242507
[Epoch:  240] cost = 0.004346909
[Epoch:  241] cost = 0.020153787
[Epoch:  242] cost = 0.0028601957
[Epoch:  243] cost = 0.00307867699
[Epoch:  244] cost = 0.0232249759
[Epoch:  245] cost = 0.00191735907
[Epoch:  246] cost = 0.00683286507
[Epoch:  247] cost = 0.0181815866
[Epoch:  248] cost = 0.00460249931
[Epoch:  249] cost = 0.00310169766
[Epoch:  250] cost = 0.0052009169
[Epoch:  251] cost = 0.00488954363
[Epoch:  252] cost = 0.00549854152
[Epoch:  253] cost = 0.00337455608
[Epoch:  254] cost = 0.00199630973
[Epoch:  255] cost = 0.0465999171
[Epoch:  256] cost = 0.00578307034
[Epoch:  257] cost = 0.00387979881
[Epoch:  258] cost = 0.00523281796
[Epoch:  259] cost = 0.00418015989
[Epoch:  260] cost = 0.00206937315
[Epoch:  261] cost = 0.000154682784
[Epoch:  262] cost = 5.09412275e-05
[Epoch:  263] cost = 0.00723975943
[Epoch:  264] cost = 0.0290901959
[Epoch:  265] cost = 0.00142

In [55]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1)==Y_test
    accuracy=correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

Accuracy: 0.847000002861023
